In [1]:
# Create the model class with init and forward functions
import csv
import os, random, sys, copy
import torch, torch.nn as nn, numpy as np
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

class sarcasmModel(nn.Module):

    def __init__(self, embedding_matrix, lstm_hidden_size=50, num_lstm_layers=1, bidirectional=True):

        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix))
        self.lstm = nn.LSTM(input_size = embedding_matrix.shape[1],
                            hidden_size = lstm_hidden_size,
                            num_layers = num_lstm_layers,
                            bidirectional = bidirectional,
                            batch_first = True)
        
        self.hidden_1 = nn.Linear(lstm_hidden_size * 2, lstm_hidden_size)
        self.hidden_2 = nn.Linear(lstm_hidden_size, 1)
        self.num_directions = 2 if bidirectional else 1
        self.relu = nn.ReLU()
    
    def forward(self, input_batch, input_lengths):
        
        embedded_input = self.embedding(input_batch)
        
        packed_input = pack_padded_sequence(embedded_input, input_lengths, batch_first=True, enforce_sorted=False)
        
        packed_output, (hn, cn) = self.lstm(packed_input)                                                                       # See docs linked below for description of hn.shape
        
        hn_view = hn.view(self.lstm.num_layers, self.num_directions, input_batch.shape[0], self.lstm.hidden_size)               # Reshape hn for clarity -- first dimension now represents each layer (total set by num_lstm_layers)
        
        hn_view_last_layer = hn_view[-1]                                                                                        # Taking the last layer for our final LSTM output
        
        hn_cat = torch.cat([hn_view_last_layer[-2, :, :], hn_view_last_layer[-1, :, :]], dim=1)                                 # Each layer has two directions. We want to use both of these vectors, so concatenate them
        
        hid = self.relu(self.hidden_1(hn_cat))
        
        output = self.hidden_2(hid)
        
        return output